In [1]:
# ======================
# 📦 INSTALLATION + TRAITEMENT COMPLET
# ======================
# Étape 1 : Installation
%pip install -q spacy scispacy openpyxl
%pip install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz
%pip install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_bc5cdr_md-0.5.4.tar.gz
%pip install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_bionlp13cg_md-0.5.4.tar.gz

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
# code optimisé pour multiprocessing

In [2]:
# 📚 IMPORTS
import spacy
import os
import zipfile
from google.colab import files
import shutil
import pandas as pd
import concurrent.futures
from tqdm import tqdm

# Configuration pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# 🧠 CHARGEMENT DES MODÈLES
try:
    nlp_models = {
        "sci": spacy.load("en_core_sci_sm"),
        "bc5cdr": spacy.load("en_ner_bc5cdr_md"),
        "bionlp": spacy.load("en_ner_bionlp13cg_md")
    }
except OSError as e:
    raise Exception(f"❌ Modèle spaCy manquant. Veuillez installer les modèles requis: {str(e)}")

# 🔍 FONCTION D'EXTRACTION MULTIMODÈLE
def extraire_entites(pmid, text):
    entites = {"PMID": pmid}
    for model_name, nlp in nlp_models.items():
        doc = nlp(str(text))
        entites[f"Entities_{model_name}"] = ", ".join(sorted(set(ent.text for ent in doc.ents)))
    return entites

# 📥 TÉLÉVERSEMENT DU ZIP
print("➡️ Téléversez un fichier ZIP contenant un dossier avec des fichiers .xlsx...")
uploaded = files.upload()
zip_filename = next(iter(uploaded))

# 📂 EXTRACTION DU ZIP
extract_dir = "extracted_files"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 🔍 RÉCUPÉRER TOUS LES FICHIERS .xlsx
excel_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(extract_dir)
    for file in files if file.lower().endswith(".xlsx")
]

if not excel_files:
    raise Exception("❌ Aucun fichier .xlsx trouvé dans le ZIP.")

# 📂 DOSSIER DE SORTIE
output_dir = "fichiers_traites"
os.makedirs(output_dir, exist_ok=True)

# 🧪 TRAITEMENT DE CHAQUE FICHIER
for path in excel_files:
    file = os.path.basename(path)
    print(f"\n📄 Traitement de : {file}")

    try:
        df = pd.read_excel(path)
        df = df.rename(columns={"Titre (TI)": "Titre", "Résumé (AB)": "Résumé"})

        if not {'PMID', 'Titre', 'Résumé'}.issubset(df.columns):
            print(f"⚠️ Le fichier {file} ne contient pas les colonnes requises. Ignoré.")
            continue

        # Combinaison des colonnes
        df["titre_resume"] = df["Titre"].fillna("") + " " + df["Résumé"].fillna("")

        # Parallélisation avec ThreadPoolExecutor et barre de progression
        entites_list = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            futures = {
                executor.submit(extraire_entites, pmid, text): idx
                for idx, (pmid, text) in enumerate(zip(df["PMID"], df["titre_resume"]))
            }

            for future in tqdm(concurrent.futures.as_completed(futures),
                             total=len(futures),
                             desc=f"🔄 Traitement de {file}"):
                try:
                    entites_list.append(future.result())
                except Exception as e:
                    print(f"⚠️ Erreur lors du traitement d'une entrée: {str(e)}")
                    continue

        # Création du DataFrame final
        df_entites = pd.DataFrame(entites_list)
        df_final = pd.merge(df, df_entites, on="PMID", how="left")

        # Sauvegarde du résultat
        output_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}_entites.xlsx")
        df_final.to_excel(output_path, index=False)
        print(f"✅ Fichier traité sauvegardé: {output_path}")

    except Exception as e:
        print(f"❌ Erreur lors du traitement du fichier {file}: {str(e)}")
        continue

# 📦 ZIPPAGE FINAL
if os.listdir(output_dir):  # Vérifie si le dossier n'est pas vide
    shutil.make_archive("resultats_entites", 'zip', output_dir)
    files.download("resultats_entites.zip")
    print("\n✅ Tous les fichiers ont été traités et sont disponibles dans resultats_entites.zip")
else:
    print("\n⚠️ Aucun fichier n'a été traité avec succès.")


/usr/local/lib/python3.11/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


➡️ Téléversez un fichier ZIP contenant un dossier avec des fichiers .xlsx...


Saving pubmed_fichier_excels.zip to pubmed_fichier_excels (4).zip

📄 Traitement de : Congo.xlsx


🔄 Traitement de Congo.xlsx: 100%|██████████| 1374/1374 [05:23<00:00,  4.24it/s]


✅ Fichier traité sauvegardé: fichiers_traites/Congo_entites.xlsx

📄 Traitement de : Egypt.xlsx


...

🔄 Traitement de Equatorial Guinea.xlsx: 100%|██████████| 21/21 [00:04<00:00,  4.61it/s]


✅ Fichier traité sauvegardé: fichiers_traites/Equatorial Guinea_entites.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Tous les fichiers ont été traités et sont disponibles dans resultats_entites.zip


In [ ]:
# 📚 IMPORTS
import spacy
import os
import zipfile
from google.colab import files
import shutil
import pandas as pd
import concurrent.futures

# Configuration pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# 🧠 CHARGEMENT DES MODÈLES
nlp_models = {
    "sci": spacy.load("en_core_sci_sm"),
    "bc5cdr": spacy.load("en_ner_bc5cdr_md"),
    "bionlp": spacy.load("en_ner_bionlp13cg_md")
}

# 🔍 FONCTION D’EXTRACTION MULTIMODÈLE
def extraire_entites(pmid, text):
    entites = {}
    for model_name, nlp in nlp_models.items():
        doc = nlp(text)
        entites[f"Entities_{model_name}"] = ", ".join(set(ent.text for ent in doc.ents))
    entites["PMID"] = pmid
    return entites

# 📥 TÉLÉVERSEMENT DU ZIP
print("➡️ Téléverse un fichier ZIP contenant un dossier avec des fichiers .xlsx…")
uploaded = files.upload()
zip_filename = next(iter(uploaded))

# 📂 EXTRACTION DU ZIP
extract_dir = "extracted_files"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 🔍 RÉCUPÉRER TOUS LES FICHIERS .xlsx
excel_files = [os.path.join(root, file)
               for root, _, files in os.walk(extract_dir)
               for file in files if file.lower().endswith(".xlsx")]

if not excel_files:
    raise Exception("❌ Aucun fichier .xlsx trouvé dans le ZIP.")

# 📂 DOSSIER DE SORTIE
output_dir = "fichiers_traites"
os.makedirs(output_dir, exist_ok=True)

# 🧪 TRAITEMENT DE CHAQUE FICHIER
for path in excel_files:
    file = os.path.basename(path)
    print(f"📄 Traitement de : {file}")

    df = pd.read_excel(path)
    df.rename(columns={"Titre (TI)": "Titre", "Résumé (AB)": "Résumé"})

    if not {'PMID', 'Titre', 'Résumé'}.issubset(df.columns):
        print(f"⚠️ Le fichier {file} ne contient pas les colonnes requises. Ignoré.")
        continue

    df["titre_resume"] = df["Titre"].astype(str) + " " + df["Résumé"].astype(str)

    # Parallélisation avec ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(extraire_entites, pmid, text)
                   for pmid, text in zip(df["PMID"], df["titre_resume"])]
        resultats = [future.result() for future in concurrent.futures.as_completed(futures)]

    df_entites = pd.DataFrame(resultats)
    df_final = pd.merge(df, df_entites, on="PMID")

    output_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}_entites.xlsx")
    df_final.to_excel(output_path, index=False)

# 📦 ZIPPAGE FINAL
shutil.make_archive("resultats_entites", 'zip', output_dir)
files.download("resultats_entites.zip")
print("✅ Tous les fichiers ont été traités et sont disponibles dans resultats_entites.zip")


➡️ Téléverse un fichier ZIP contenant un dossier avec des fichiers .xlsx…


Saving pubmed_fichier_excels.zip to pubmed_fichier_excels (2).zip
📄 Traitement de : Congo.xlsx
⚠️ Le fichier Congo.xlsx ne contient pas les colonnes requises. Ignoré.
📄 Traitement de : Egypt.xlsx
⚠️ Le fichier Egypt.xlsx ne contient pas les colonnes requises. Ignoré.
📄 Traitement de : Libya.xlsx
⚠️ Le fichier Libya.xlsx ne contient pas les colonnes requises. Ignoré.
📄 Traitement de : Mozambique.xlsx
⚠️ Le fichier Mozambique.xlsx ne contient pas les colonnes requises. Ignoré.
📄 Traitement de : Niger.xlsx
⚠️ Le fichier Niger.xlsx ne contient pas les colonnes requises. Ignoré.
📄 Traitement de : Mauritius.xlsx
⚠️ Le fichier Mauritius.xlsx ne contient pas les colonnes requises. Ignoré.
📄 Traitement de : Malawi.xlsx
⚠️ Le fichier Malawi.xlsx ne contient pas les colonnes requises. Ignoré.
📄 Traitement de : Madagascar.xlsx
⚠️ Le fichier Madagascar.xlsx ne contient pas les colonnes requises. Ignoré.
📄 Traitement de : Ivory Coast.xlsx
⚠️ Le fichier Ivory Coast.xlsx ne contient pas les colonnes r

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Tous les fichiers ont été traités et sont disponibles dans resultats_entites.zip


In [ ]:
##  code initiale

In [ ]:
# 📚 IMPORTS
import spacy

import os
import zipfile
from google.colab import files
import shutil
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# 🧠 CHARGEMENT DES MODÈLES
nlp_sci = spacy.load("en_core_sci_sm")
nlp_bc5cdr = spacy.load("en_ner_bc5cdr_md")
nlp_bionlp = spacy.load("en_ner_bionlp13cg_md")

# 🔍 FONCTION D’EXTRACTION MULTIMODÈLE
def extraire_entites(pmid, text):
    doc_sci = nlp_sci(text)
    doc_bc5cdr = nlp_bc5cdr(text)
    doc_bionlp = nlp_bionlp(text)

    ent_sci = ", ".join(set(ent.text for ent in doc_sci.ents))
    ent_bc5 = ", ".join(set(ent.text for ent in doc_bc5cdr.ents))
    ent_bio = ", ".join(set(ent.text for ent in doc_bionlp.ents))

    return {
        "PMID": pmid,
        "Entities_sci_sm": ent_sci,
        "Entities_bc5cdr": ent_bc5,
        "Entities_bionlp": ent_bio
    }

# 📥 TÉLÉVERSEMENT DU ZIP
print("➡️ Téléverse un fichier ZIP contenant un dossier avec des fichiers .xlsx…")
uploaded = files.upload()
zip_filename = list(uploaded.keys())[0]

# 📂 EXTRACTION DU ZIP
extract_dir = "extracted_files"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 🔍 RÉCUPÉRER TOUS LES FICHIERS .xlsx
excel_files = []
for root, dirs, files_ in os.walk(extract_dir):
    for file in files_:
        if file.lower().endswith(".xlsx"):
            excel_files.append(os.path.join(root, file))

if not excel_files:
    raise Exception("❌ Aucun fichier .xlsx trouvé dans le ZIP.")

# 📂 DOSSIER DE SORTIE
output_dir = "fichiers_traites"
os.makedirs(output_dir, exist_ok=True)

# 🧪 TRAITEMENT DE CHAQUE FICHIER
for path in excel_files:
    file = os.path.basename(path)
    print(f"📄 Traitement de : {file}")

    df = pd.read_excel(path)

    if not {'PMID', 'Titre', 'Résumé'}.issubset(df.columns):
        print(f"⚠️ Le fichier {file} ne contient pas les colonnes requises. Ignoré.")
        continue

    df["titre_resume"] = df["Titre"].astype(str) + " " + df["Résumé"].astype(str)

    resultats = [
        extraire_entites(pmid, text)
        for pmid, text in zip(df["PMID"], df["titre_resume"])
    ]

    df_entites = pd.DataFrame(resultats)
    df_final = pd.merge(df, df_entites, on="PMID")

    nom_sans_ext = os.path.splitext(file)[0]
    output_path = os.path.join(output_dir, f"{nom_sans_ext}_entites.xlsx")
    df_final.to_excel(output_path, index=False)

# 📦 ZIPPAGE FINAL
shutil.make_archive("resultats_entites", 'zip', output_dir)
files.download("resultats_entites.zip")
print("✅ Tous les fichiers ont été traités et sont disponibles dans resultats_entites.zip")

/usr/local/lib/python3.11/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


➡️ Téléverse un fichier ZIP contenant un dossier avec des fichiers .xlsx…


Saving pubmed_result.zip to pubmed_result.zip
📄 Traitement de : Tanzania.xlsx
📄 Traitement de : Zimbabwe.xlsx
📄 Traitement de : South Africa.xlsx
📄 Traitement de : Togo.xlsx
📄 Traitement de : South Africa1.xlsx
📄 Traitement de : Madagascar.xlsx
📄 Traitement de : Algeria.xlsx
📄 Traitement de : Guine Bissau.xlsx
📄 Traitement de : Cameroon.xlsx
📄 Traitement de : Equatorial Guinea.xlsx
📄 Traitement de : Malawi.xlsx
📄 Traitement de : South Sudan.xlsx
📄 Traitement de : Sierra Leone.xlsx
📄 Traitement de : Angola.xlsx
📄 Traitement de : Mozambique.xlsx
📄 Traitement de : Zambia.xlsx
📄 Traitement de : Senegal.xlsx
📄 Traitement de : Burundi.xlsx
📄 Traitement de : Morocco.xlsx
📄 Traitement de : Botswana.xlsx
📄 Traitement de : Congo.xlsx
📄 Traitement de : Ivory Coast.xlsx
📄 Traitement de : Ethiopia.xlsx
📄 Traitement de : Nigeria.xlsx
📄 Traitement de : Seychelles.xlsx
📄 Traitement de : Djibouti.xlsx
📄 Traitement de : Burkina Faso.xlsx
📄 Traitement de : Sudan.xlsx
📄 Traitement de : Liberia.xlsx
📄 Trai

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Tous les fichiers ont été traités et sont disponibles dans resultats_entites.zip
